[current dir](.)

In [7]:
filename = 'inputs/Day16.txt'
lines = open(filename).read().splitlines()

p1 = lines[:3172]
p2 = lines[3174:]
print(p1[:10], "...", p1[-10:], sep="\n")
print(p2[:10], "...", p2[-10:], sep="\n")

['Before: [3, 0, 1, 3]', '15 2 1 3', 'After:  [3, 0, 1, 1]', '', 'Before: [1, 3, 2, 0]', '11 2 2 0', 'After:  [4, 3, 2, 0]', '', 'Before: [0, 3, 3, 1]', '14 3 2 0']
...
['After:  [0, 0, 2, 4]', '', 'Before: [2, 3, 1, 2]', '9 2 1 3', 'After:  [2, 3, 1, 1]', '', 'Before: [0, 3, 1, 1]', '6 0 0 3', 'After:  [0, 3, 1, 0]', '']
['1 0 0 1', '4 1 1 1', '14 0 0 3', '14 3 2 2', '12 3 2 1', '1 1 2 1', '11 1 0 0', '3 0 2 1', '14 2 2 0', '9 0 2 0']
...
['1 2 3 2', '11 2 1 1', '3 1 1 3', '14 2 2 2', '14 2 2 1', '14 1 3 0', '3 0 2 1', '1 1 2 1', '11 3 1 3', '3 3 2 0']


Part 1

In [118]:
import re
    
def part1():
    match3 = 0
    
    for i in range(0, len(p1), 4):
        beforeS, instS, afterS = p1[i:i+3]
        m = re.search('^Before: \[(\d), (\d), (\d), (\d)\]$', beforeS)
        before = [int(v) for v in m.groups()]
        m = re.search('^(\d+) (\d+) (\d+) (\d+)$', instS)
        inst = [int(v) for v in m.groups()]
        m = re.search('^After:  \[(\d), (\d), (\d), (\d)\]$', afterS)
        after = [int(v) for v in m.groups()]
        
        matching = {
            'addr': after[inst[3]] == (before[inst[1]] + before[inst[2]]),
            'addi': after[inst[3]] == (before[inst[1]] + inst[2]),
            'mulr': after[inst[3]] == (before[inst[1]] * before[inst[2]]),
            'muli': after[inst[3]] == (before[inst[1]] * inst[2]),
            'banr': after[inst[3]] == (before[inst[1]] & before[inst[2]]),
            'bani': after[inst[3]] == (before[inst[1]] & inst[2]),
            'borr': after[inst[3]] == (before[inst[1]] | before[inst[2]]),
            'bori': after[inst[3]] == (before[inst[1]] | inst[2]),
            'setr': after[inst[3]] == before[inst[1]],
            'seti': after[inst[3]] == inst[1],
            'gtir': after[inst[3]] == (inst[1] > before[inst[2]]),
            'gtri': after[inst[3]] == (before[inst[1]] > inst[2]),
            'gtrr': after[inst[3]] == (before[inst[1]] > before[inst[2]]),
            'eqir': after[inst[3]] == (inst[1] == before[inst[2]]),
            'eqri': after[inst[3]] == (before[inst[1]] == inst[2]),
            'eqrr': after[inst[3]] == (before[inst[1]] == before[inst[2]]) 
        }
        if sum(matching.values()) >= 3:
            match3 += 1
                
    return match3
    
part1()

493

Part 2

In [70]:
import re

instructions = {
    'addr': lambda registers, a, b: registers[a] + registers[b], 
    'addi': lambda registers, a, b: registers[a] + b,
    'mulr': lambda registers, a, b: registers[a] * registers[b], 
    'muli': lambda registers, a, b: registers[a] * b,
    'banr': lambda registers, a, b: registers[a] & registers[b], 
    'bani': lambda registers, a, b: registers[a] & b,
    'borr': lambda registers, a, b: registers[a] | registers[b], 
    'bori': lambda registers, a, b: registers[a] | b,
    'setr': lambda registers, a, b: registers[a], 
    'seti': lambda registers, a, b: a,
    'gtir': lambda registers, a, b: int(a > registers[b]), 
    'gtri': lambda registers, a, b: int(registers[a] > b), 
    'gtrr': lambda registers, a, b: int(registers[a] > registers[b]),
    'eqir': lambda registers, a, b: int(a == registers[b]), 
    'eqri': lambda registers, a, b: int(registers[a] == b), 
    'eqrr': lambda registers, a, b: int(registers[a] == registers[b])
}

def part2():
    # Process matches between opcodes and cases
    matches = {opcode: [i for i in range(len(instructions))] for opcode in instructions.keys()}
    for i in range(0, len(p1), 4):
        beforeS, instS, afterS = p1[i:i+3]
        m = re.search('^Before: \[(\d), (\d), (\d), (\d)\]$', beforeS)
        before = [int(v) for v in m.groups()]
        m = re.search('^(\d+) (\d+) (\d+) (\d+)$', instS)
        inst = [int(v) for v in m.groups()]
        m = re.search('^After:  \[(\d), (\d), (\d), (\d)\]$', afterS)
        after = [int(v) for v in m.groups()]
        for opcode in instructions.keys():
            if instructions[opcode](before, inst[1], inst[2]) != after[inst[3]] and inst[0] in matches[opcode]:
                    matches[opcode].remove(inst[0])
              
    # Resolve matches
    solvedinst = {}
    singles = {o: v[0] for o, v in matches.items() if len(v) == 1 and o not in solvedinst.keys()}
    while len(singles) > 0:
        matches = {i:v for i,v in matches.items() if i not in singles.keys()}
        for opcode, finalvalue in singles.items():
            solvedinst[finalvalue] = opcode
            matches = {i: [p for p in possible if finalvalue != p] for i, possible in matches.items()}
        singles = {o: v[0] for o, v in matches.items() if len(v) == 1 and o not in solvedinst.keys()}
    
    # Simulate the program
    registers = [0, 0, 0, 0]
    for line in p2:
        m = re.search('^(\d+) (\d+) (\d+) (\d+)$', line)
        ix = [int(v) for v in m.groups()]
        opc = solvedinst[ix[0]]
        registers[ix[3]] = instructions[opc](registers, ix[1], ix[2])
    print(registers[0])
    
part2()

445
